## import包

In [1]:
import sys
sys.path.append('../AutoX')

In [2]:
from autox import AutoXRecommend

/home/caihengxing/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/home/caihengxing/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/caihengxing/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not mo

In [3]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

## 读取数据

In [4]:
path = '~/AutoX/autox/autox_recommend/datasets/MovieLens_AutoX/'

In [5]:
inter_df = pd.read_csv(path + 'inter_df.csv')
item_df = pd.read_csv(path + 'item_df.csv')
test = pd.read_csv(path + 'test.csv')

## 配置参数

In [6]:
uid = 'userId'
iid = 'movieId'
time_col = 'time'
recall_num = 100

## 准备测试集结果

In [7]:
inter_df[time_col].min(), inter_df[time_col].max()

('1995-01-09 11:46:49', '2019-11-14 23:20:55')

In [8]:
test[time_col].min(), test[time_col].max()

('2019-11-15 00:08:42', '2019-11-21 09:06:53')

In [9]:
assert(test[time_col].min() > inter_df[time_col].max())

In [10]:
positive_items_test = test.groupby([uid])[iid].apply(list)
test_users = positive_items_test.keys()
test_items = []

for i, user in tqdm(enumerate(test_users)):
    test_items.append(positive_items_test[user])
    
print("Total users in testidation:", len(test_users))

283it [00:00, 177692.82it/s]

Total users in testidation: 283


## 时间列转化

In [11]:
inter_df[time_col] = pd.to_datetime(inter_df[time_col])

In [12]:
inter_df.head()

,userId,movieId,time
0,1,296,2006-05-17 15:34:04
1,1,307,2006-05-17 12:27:08
2,1,665,2006-05-17 15:13:40
3,1,1237,2006-05-17 12:27:19
4,1,2632,2006-05-17 15:04:08


## 执行AutoX

In [13]:
autoXRecommend = AutoXRecommend()

autoXRecommend.fit(inter_df = inter_df, user_df = None, item_df = item_df,
                  uid = uid, iid = iid, time_col = time_col,
                  recall_num = recall_num,
                  time_decay = 0.99,
                  debug = True, debug_save_path = './temp_MovieLens')


popular_recall
train
2019-11-01 00:21:32 2019-11-07 23:45:48


100%|██████████| 331/331 [00:00<00:00, 23444.92it/s]


HIT:  0.27881950171515285
valid
2019-11-08 00:01:28 2019-11-14 23:20:55


100%|██████████| 310/310 [00:00<00:00, 1136.14it/s]


HIT:  0.27728749446707573

history_recall
train
2019-11-01 00:21:32 2019-11-07 23:45:48


...

ItemCF recommend


100%|██████████| 213/213 [01:51<00:00,  1.92it/s]


(21300, 3)
ItemCF recall:  (21300, 4)
mean: 0.002347417840375587
sum: 50.0

binary_recall


100%|██████████| 310/310 [01:44<00:00,  2.96it/s]


(21300, 3)
BinaryNet recall:  (21300, 4)
0.0032863849765258214

merge recalls

feature engineer
customer feature engineer
interact feature engineer
train_fe shape: (71125, 1147)

ranker
defaultdict(<class 'dict'>, {'valid_0': {'map@12': 0.5546064728628005}})
                          feature  importance
2                      n_purchase         314
1140     latest_purchase_time_sub         264
3              n_purchase_nunique         121
1                    binary_score         113
1132  purchase_corr_item_max_time          94
0                    itemcf_score          73
1141                  movieId_idx          67
1133       purchase_corr_item_cnt          35
24                         tag_21          31
995                       tag_992          21
210                       tag_207          12
75                         tag_72          12
340                       tag_337          10
50                         tag_47          10
1127                     tag_1124           9
732  

In [14]:
res = autoXRecommend.transform(test_users)


popular recall, test
2019-11-08 00:01:28 2019-11-14 23:20:55


100%|██████████| 283/283 [00:00<00:00, 45743.11it/s]


history recall, test

itemcf recall, test



100%|██████████| 202/202 [02:25<00:00,  1.39it/s]



binary recall, test


100%|██████████| 283/283 [01:31<00:00,  3.10it/s]



merge recalls

feature engineer
customer feature engineer
interact feature engineer
test_fe shape: (66920, 1146)

inference
[1/1]
(66920, 1146)


## 查看结果

In [15]:
def apk(actual, predicted, k=12):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=12):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [17]:
outputs = res['prediction']
print("mAP Score on Validation set:", mapk(test_items, outputs))

mAP Score on Validation set: 0.08030425675382308
